## **Trabajo Práctico Individual n° 2: Aprendizaje Supervisado**
____
__Universidad Tecnológica Nacional, Buenos Aires__\
__Ingeniería Industrial__\
__Ciencia de Datos - Curso I5521__
____

- Nombre: Rodrigo
- Apellido: Perez Garofalo
- Legajo: 1632589

En este trabajo práctico aplicaremos algunas técnicas aprendidas para el aprendizaje automático supervisado.

Utilizaremos un dataset de airbnb donde buscaremos predecir el precio de las noches de cada hogar en funcion de algunas variables.

#### Variables:
- Unnamed: 0: Variable de contador de registros (no será utilizada en nuetro análisis)
- neighbourhood: Barrio donde se encuentra ubicado la casa.
- room_type: Tipo de hogar en alquiler.
- price: Precio por noche.
- minimum_nights: Cantidad minimas de noches que pueden alquilar el hogar.
- number_of_reviews: Cantidad de reviews
- reviews_per_month: Tasa de reviews que tienen por mes
- calculated_host_listings_count: Cantidad de hogares ofrecidos por el host.
- availability_365: Cantidad de dias que se encuentra en alquiler el hogar

## **Objetivos del trabajo practico**
- resolver los requisitos que se detallan en las proximas celdas
- dejar comentarios # en cada linea de codigo que se agregue
- fecha limite de entrega: 27 de Octubre 2022 a las 23h59

In [1]:
## Importo librerías de maniupulación de datos
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
## Importo librerías de Aprendizaje automático
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV

##### **Objetivo 1**
Lo primero que deberan hacer será importar
- el dataset bajo en nombre "london"
- corroborar que lo hayamos cargado correctamente.

In [3]:
## guardamos el dataset en la variable "London"
london = pd.read_csv('TP2_airbnb_london.csv')
london.head()

,Unnamed: 0,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,0,Haringey,Private room,35,1,131,1.71,4,262
1,1,Ealing,Private room,70,2,2,0.07,6,62
2,2,Islington,Private room,45,1,14,0.14,2,364
3,5,Tower Hamlets,Entire home/apt,102,5,7,0.07,56,295
4,6,Barnet,Private room,29,5,114,1.13,3,285


##### **Objetivo 2**
Ahora imprimir también las dimensiones del dataset y eliminar la variable "Unnamed:0" del dataset.
Volver a guardar el dataset sin esta variable bajo el nombre "london" 

In [4]:
## Imprimimos las dimensiones del dataset
np.shape(london)

(51827, 9)

In [5]:
## Eliminamos la variable "Unnamed: 0"
london = london.drop('Unnamed: 0', axis=1)

##### **Objetivo 3**
Corroborar que el dataset no cuenta con valores nulos, si los hubiera llenarlos todos con 0.

In [6]:
## Imprimimos la información del dataset
london.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51827 entries, 0 to 51826
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   neighbourhood                   51827 non-null  object 
 1   room_type                       51827 non-null  object 
 2   price                           51827 non-null  int64  
 3   minimum_nights                  51827 non-null  int64  
 4   number_of_reviews               51827 non-null  int64  
 5   reviews_per_month               51827 non-null  float64
 6   calculated_host_listings_count  51827 non-null  int64  
 7   availability_365                51827 non-null  int64  
dtypes: float64(1), int64(5), object(2)
memory usage: 3.2+ MB


In [7]:
## Vuelvo a chequear que no haya ningún valor nulo
london.isnull().sum().sum()

0

##### **Objetivo 4**
Mostrar el tipo de dato para las variables "neighbourhood" y "room_type".

In [8]:
# Imprimimos el data type de las variables "neighbourhood" y "room_type"
print(london.neighbourhood.dtypes)
print(london.room_type.dtypes)

object
object


##### **Objetivo 5**
Generar las variables dummies para estas dos variables categóricas y unirlas al dataset london.


In [9]:
## Generamos las dummies para la variable neighbourhood, dentro de la variable neigh_dummies.
neigh_dummies = pd.get_dummies(pd.Series(london['neighbourhood']))

Imprimir las dimensiones y primeras lineas de estas dummies

In [10]:
# Imprimimos las dimensiones de los dummies generados para neighbourhood
np.shape(neigh_dummies)

(51827, 33)

In [11]:
# Imprimimos las primeras lineas del dummie
neigh_dummies.head()

,Barking and Dagenham,Barnet,Bexley,Brent,Bromley,Camden,City of London,Croydon,Ealing,Enfield,...,Merton,Newham,Redbridge,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
## Generamos las dummies para la variable room_type, dentro de la variable room_dummies.
room_dummies = pd.get_dummies(pd.Series(london['room_type']))

Imprimir las dimensiones y primeras lineas de estas dummies

In [13]:
## Imprimimos las dimensiones de los dummies generados para room_types
np.shape(room_dummies)

(51827, 3)

In [14]:
# Imprimimos las primeras lineas del dummie
room_dummies.head()

,Entire home/apt,Private room,Shared room
0,0,1,0
1,0,1,0
2,0,1,0
3,1,0,0
4,0,1,0


##### Dividiremos nuestro dataset en la variable dependiente "y" (label o etiqueta) y las variables independientes "x"

In [15]:
## Hacemos un join con los dummies creados previamente
london = london.join(room_dummies).join(neigh_dummies)

In [16]:
## Guardamos el target en la variable dependiente "y" y las variables independientes "x"
y = np.array(london[["price"]])
x = london.drop(['price', 'neighbourhood','room_type'], axis=1)

##### **Objetivo 7**

Dividir el dataset entre Train y Test, tomar un test_size del 30%.

In [17]:
## Dividimos el dataset entre test y train
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

##### **Objetivo 8**
Con el StandardScaler generado en la linea siguiente, realizar el transform para el xtrain y xtest guardandolos en nuevas variables llamadas "xtrain_scal" y "xtest_scal".


In [18]:
## Separo los datasets de train y test de los dummies para hacer el scaling
x_train_nodummies = x_train.iloc[:,:5]
x_test_nodummies= x_test.iloc[:,:5]
## Los dummies los convertimos en numpy arrays para poder usar np.concatenate mas adelante
x_train_dummies = x_train.iloc[:,5:].to_numpy()
x_test_dummies = x_test.iloc[:,5:].to_numpy()

In [19]:
## Ajustamos el scaler al dataset
scaler = preprocessing.StandardScaler().fit(x_train_nodummies)

In [20]:
## Scaling al dataset de train
xtrain_scal = scaler.transform(x_train_nodummies)

In [21]:
## Scaling al dataset de test
xtest_scal = scaler.transform(x_test_nodummies)

In [22]:
## Volvemos a unir las features a las cuales les hicimos scaling con las dummies
xtrain_scal = np.concatenate((xtrain_scal,x_train_dummies),axis=1)
xtest_scal = np.concatenate((xtest_scal,x_test_dummies),axis=1)

##### **Objetivo 9**
Crear un modelo lineal y entrenarlo para nuestro dataset.

Obtener las predicciones y el "mean_squared_error" para el test set.

In [23]:
## Guardamos el modelo lineal en la variable linear
linear = LinearRegression()

In [24]:
## Entrenamos al modelo con los datos de entrenamiento
linear.fit(xtrain_scal,y_train)

LinearRegression()

In [25]:
## Hacemos una predicción con los datos de test
prediction = linear.predict(xtest_scal)
print(prediction,y_test)

[[ 44.13085938]
 [106.10742188]
 [108.1015625 ]
 ...
 [127.97460938]
 [152.79101562]
 [128.65039062]] [[ 65]
 [ 75]
 [110]
 ...
 [100]
 [150]
 [101]]


In [26]:
## Mean squared error de las prediciones
mean_squared_error(y_test,prediction)

1505.676164841129

##### **Objetivo 10**
Crear un SVM Regressor y un GridSearch para realizar la búsqueda de hiperparámetros.

Asignar dos posibles valores para el parámetro "kernel", y tres posibles valores para "C" y "gamma".

Setear al parámetro "refit" en True y hacer 5 CrossValidation.

In [27]:
## Seleccionamos el modelo Support Vector Regressor
model = SVR()

In [28]:
## Seteamos los posibles valores del modelo
model_param = {'kernel':('linear', 'rbf'), 'C':[1, 10 , 20], 'gamma':[ 0.1,0.5,1]}

In [29]:
np.shape(xtrain_scal)

(36278, 41)

In [30]:
## Ejecutamos un gridsearch para encontrar los mejores hiperparametros
regressor_svr = GridSearchCV(model, model_param, cv=5, verbose=True, refit = True)

In [31]:
## Fiteamos el modelo a los datos de entrenamiento
regressor_svr.fit(xtrain_scal, y_train.ravel())

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [1, 10, 20], 'gamma': [0.1, 0.5, 1],
                         'kernel': ('linear', 'rbf')},
             verbose=True)

##### **Objetivo 11**

Mostrar cuales fueron los mejores hiperparámetros y para estos obtener el mean_squared_error en el test set.

In [32]:
## Imprimimos los mejores hiperparametros
regressor_svr.best_params_

{'C': 20, 'gamma': 0.1, 'kernel': 'rbf'}

In [33]:
## Imprimimos el mejor score
regressor_svr.best_score_

0.5042187417531137

In [34]:
## Realizamos la predicción con los datos de test
prediction2 = regressor_svr.predict(xtest_scal)
prediction2

array([ 41.08387545,  91.12204563,  90.02355149, ..., 110.23405191,
       141.0063984 , 121.29109033])

In [35]:
mean_squared_error(y_test,prediction2)

1498.2037579655203

##### **Objetivo 12**
Repetir el objetivo 10 pero ahora seteando nuevos valores para los hiperparámetros mencionados con el objetivo de mejorar las predicciones.

Cuales son los mejores hiperparámetros ahora?
Obtenemos una mejora en las predicciones?
Mostrar los resultados.

In [36]:
## Seleccionamos el modelo Support Vector Regressor
model2 = SVR()

In [37]:
## Seteamos los posibles valores del modelo
model_param2 = {'kernel':('poly', 'rbf'), 'C':[20, 30, 40], 'gamma':[ 0.01,0.05,0.1]}

In [38]:
## Ejecutamos un gridsearch para encontrar los mejores hiperparametros
regressor_svr2 = GridSearchCV(model2, model_param2, cv=5, verbose=True, refit = True)

In [39]:
## Fiteamos el modelo a los datos de entrenamiento
regressor_svr2.fit(xtrain_scal, y_train.ravel())

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [20, 30, 40], 'gamma': [0.01, 0.05, 0.1],
                         'kernel': ('poly', 'rbf')},
             verbose=True)

In [40]:
## Imprimimos los mejores hiperparametros
regressor_svr2.best_params_

{'C': 40, 'gamma': 0.1, 'kernel': 'rbf'}

In [41]:
## Realizamos la predicción con los datos de test
prediction3 = regressor_svr2.predict(xtest_scal)
prediction3

array([ 41.72437555,  91.55932738,  90.07874958, ..., 109.78414109,
       141.57058685, 122.03569953])

In [42]:
## Calculamos el Mean Square Error
mean_squared_error(y_test,prediction3)

1491.397940383268